In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
restaurants = pd.concat([iv, sb], ignore_index=True)

NameError: name 'iv' is not defined

In [ ]:
restaurants['Takeout']

,Takeout
0,"pickup, delivery"
1,"pickup, delivery"
2,NaN
3,"pickup, delivery"
4,"pickup, delivery"
...,...
95,NaN
96,"pickup, delivery"
97,NaN
98,delivery


In [ ]:
# string split takeouts into list objects
restaurants['Takeout'] = restaurants['Takeout'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

In [ ]:
restaurants

,Business_ID,Name,NumReviews,Rating,Cuisine,Cuisine1,Cuisine2,PriceRange,Takeout,Waitlist_Needed,...,Sa_end,Latitude,Longitude,Address,City,State,Zip,Menu,Description,descriptors
0,j6t0BW13CZOiTfptbJkyUg,Zocalo,128.0,4.8,Mexican,Coffee & Tea,Breakfast & Brunch,$$,"[pickup, delivery]",NaN,...,2130.0,34.413034,-119.857311,901 Embarcadero Del Mar,Isla Vista,CA,93117,https://www.zocaloiv.com/menu,,Mexican Coffee & Tea Breakfast & Brunch
1,Fz-BIAOoPIzfUj2NdFPEqA,Santa Barbara Fish Market and Restaurant - Goleta,97.0,4.4,Seafood Markets,Seafood,Fish & Chips,$$,"[pickup, delivery]",False,...,1900.0,34.428633,-119.876655,7127 Hollister Ave,Goleta,CA,93117,https://mcusercontent.com/cdc7cd01d9f395f91417...,,Seafood Markets Seafood Fish & Chips
2,Rl42JbSMsmNW3LRjsTMYAg,Freebirds,1423.0,4.1,Mexican,,,$$,[],NaN,...,2300.0,34.413300,-119.855690,879 Embarcadero Del Norte,Isla Vista,CA,93117,NaN,,Mexican
3,eGI_WepIIjYahIrMVRknfw,Sizzling Lunch,148.0,4.2,Japanese,Steakhouses,Asian Fusion,$$,"[pickup, delivery]",NaN,...,2100.0,34.410924,-119.855268,6533 Trigo Rd,Goleta,CA,93117,https://www.sizzlinglunch.com/menu,,Japanese Steakhouses Asian Fusion
4,9xW7LsJpyhVZFRb6z9xorg,Ca' Dario - Goleta,282.0,4.0,Italian,Pizza,Wine Bars,$$,"[pickup, delivery]",NaN,...,2100.0,34.429140,-119.869020,250 Storke Rd,Goleta,CA,93117,NaN,,Italian Pizza Wine Bars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0ac5lGA_0wdKDnib3y59Xg,Flor De Maiz,860.0,4.1,Cocktail Bars,,Mexican,$$$,[],True,...,2100.0,34.413192,-119.688438,29 E Cabrillo Blvd,Santa Barbara,CA,93101,https://flordemaizsb.com/menu,,Cocktail Bars Mexican
96,r926yelr_EJAKQCjdD6AWA,State & Fig,573.0,4.0,Cafes,,New American,$$,"[pickup, delivery]",NaN,...,NaN,34.423044,-119.703208,1114 State St,Santa Barbara,CA,93101,https://2abb8d04-4c1a-462d-83dd-502e5983f4a0.f...,Tapas/Small Plates,Cafes New American Tapas/Small Plates
97,TfxALXy2PdjCCt3GsnsV-Q,Tamar,46.0,4.6,Middle Eastern,,Vegan,NaN,[],NaN,...,NaN,34.415824,-119.689954,121 E Yanonali St,Santa Barbara,CA,93101,NaN,Vegetarian,Middle Eastern Vegan Vegetarian
98,FEBwxZMEpxRwVXx9Fm0XZw,Chad's,1727.0,3.9,Breakfast & Brunch,,Salad,$$,[delivery],NaN,...,1430.0,34.409922,-119.692139,216 W Cabrillo Blvd,Santa Barbara,CA,93101,https://sambosrestaurant.com/menu-breakfast-lu...,American,Breakfast & Brunch Salad American


In [ ]:
# one-hot encode takeout cateogires
takeout_encoded = restaurants['Takeout'].apply(lambda x: pd.Series(1, index=x)).fillna(0)

In [ ]:
restaurants = pd.concat([restaurants, takeout_encoded], axis=1)

In [ ]:
restaurants['PriceRange'].unique()

def clean_price(x):
    if x == '$':
        return 'cheap' # budget friendly, affordable
    elif x == '$$':
        return 'moderate' # mid-range
    elif x == '$$$':
        return 'high' # high-end, upscale, fine-dining
    elif x == '$$$$':
        return 'expensive' # luxury, premium experience

In [ ]:
restaurants['price_descrip'] = restaurants['PriceRange'].apply(clean_price)

In [ ]:
def is_open(start, end, meal_start, meal_end):
    # Normalize start/end times to the range of minutes from midnight
    start_minutes = (start // 100) * 60 + (start % 100)
    end_minutes = (end // 100) * 60 + (end % 100)
    meal_start_minutes = (meal_start // 100) * 60 + (meal_start % 100)
    meal_end_minutes = (meal_end // 100) * 60 + (meal_end % 100)

    # Check if the restaurant's hours overlap with the meal time
    return 1 if (start_minutes < meal_end_minutes and end_minutes > meal_start_minutes) else 0

# Meal time windows (in 24-hour format)
meal_times = {
    'Breakfast': (700, 1000),
    'Lunch': (1100, 1400),
    'Dinner': (1700, 2100)
}

# Apply the function for each day of the week
for day in ['Su', 'M', 'T', 'W', 'R', 'F', 'Sa']:
    for meal, (meal_start, meal_end) in meal_times.items():
        # Create new columns indicating whether the restaurant is open during each meal time
        restaurants[f'{day}_{meal}'] = restaurants.apply(lambda x: is_open(x[f'{day}_start'], x[f'{day}_end'], meal_start, meal_end), axis=1)

In [ ]:
restaurants['Cuisine'] = restaurants['Cuisine'].fillna('')
restaurants['Cuisine1'] = restaurants['Cuisine1'].fillna('')
restaurants['Cuisine2'] = restaurants['Cuisine2'].fillna('')
restaurants['Description'] = restaurants['Description'].fillna('')

restaurants['descriptors'] = restaurants.apply(lambda x: str(x['Cuisine']).strip() + ' ' + str(x['Cuisine1']).strip() + ' ' + str(x['Cuisine2']).strip() + ' ' + str(x['Description']).strip(), axis=1)

In [ ]:
restaurants.columns

Index(['Business_ID', 'Name', 'NumReviews', 'Rating', 'Cuisine', 'Cuisine1',
       'Cuisine2', 'PriceRange', 'Takeout', 'Waitlist_Needed', 'Su_start',
       'Su_end', 'M_start', 'M_end', 'T_start', 'T_end', 'W_start', 'W_end',
       'R_start', 'R_end', 'F_start', 'F_end', 'Sa_start', 'Sa_end',
       'Latitude', 'Longitude', 'Address', 'City', 'State', 'Zip', 'Menu',
       'Description', 'descriptors', 'pickup', 'delivery',
       'restaurant_reservation', 'price_descrip', 'Su_Breakfast', 'Su_Lunch',
       'Su_Dinner', 'M_Breakfast', 'M_Lunch', 'M_Dinner', 'T_Breakfast',
       'T_Lunch', 'T_Dinner', 'W_Breakfast', 'W_Lunch', 'W_Dinner',
       'R_Breakfast', 'R_Lunch', 'R_Dinner', 'F_Breakfast', 'F_Lunch',
       'F_Dinner', 'Sa_Breakfast', 'Sa_Lunch', 'Sa_Dinner'],
      dtype='object')

In [ ]:
restaurants_cleaned = pd.DataFrame({'business_id': restaurants['Business_ID'],
                                    'name': restaurants['Name'],
                                    'num_reviews': restaurants['NumReviews'],
                                    'rating': restaurants['Rating'],
                                    'descriptors': restaurants['descriptors'],
                                    'price_descrip': restaurants['price_descrip'],
                                    'su_breakfast': restaurants['Su_Breakfast'],
                                    'su_lunch': restaurants['Su_Lunch'],
                                    'su_dinner': restaurants['Su_Dinner'],
                                    'm_breakfast': restaurants['M_Breakfast'],
                                    'm_lunch': restaurants['M_Lunch'],
                                    'm_dinner': restaurants['M_Dinner'],
                                    't_breakfast': restaurants['T_Breakfast'],
                                    't_lunch': restaurants['T_Lunch'],
                                    't_dinner': restaurants['T_Dinner'],
                                    'w_breakfast': restaurants['W_Breakfast'],
                                    'w_lunch': restaurants['W_Lunch'],
                                    'w_dinner': restaurants['W_Dinner'],
                                    'r_breakfast': restaurants['R_Breakfast'],
                                    'r_lunch': restaurants['R_Lunch'],
                                    'r_dinner': restaurants['R_Dinner'],
                                    'f_breakfast': restaurants['F_Breakfast'],
                                    'f_lunch': restaurants['F_Lunch'],
                                    'f_dinner': restaurants['F_Dinner'],
                                    'sa_breakfast': restaurants['Sa_Breakfast'],
                                    'sa_lunch': restaurants['Sa_Lunch'],
                                    'sa_dinner': restaurants['Sa_Dinner'],
                                    'pickup': restaurants['pickup'],
                                    'delivery': restaurants['delivery'],
                                    'restaurant_reservation': restaurants['restaurant_reservation']})

In [ ]:
restaurants_cleaned.to_csv('restaurants_cleaned.csv', index=False)
files.download('restaurants_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>